## Nima Samadi
## Deep Learning HW3 - P2

In [1]:
import os
IS_COLAB_BACKEND = 'COLAB_GPU' in os.environ  # this is always set on Colab, the value is 0 or 1 depending on GPU presence
if IS_COLAB_BACKEND:
  from google.colab import auth
  # Authenticates the Colab machine and also the TPU using your
  # credentials so that they can access your private GCS buckets.
  auth.authenticate_user()

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
import seaborn as sn
import datetime
from sklearn.metrics import confusion_matrix

# Detect hardware
try:
  tpu_resolver = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
except ValueError:
  tpu_resolver = None
  gpus = tf.config.experimental.list_logical_devices("GPU")

# Select appropriate distribution strategy
if tpu_resolver:
  tf.config.experimental_connect_to_cluster(tpu_resolver)
  tf.tpu.experimental.initialize_tpu_system(tpu_resolver)
  strategy = tf.distribute.experimental.TPUStrategy(tpu_resolver)
  print('Running on TPU ', tpu_resolver.cluster_spec().as_dict()['worker'])
elif len(gpus) > 1:
  strategy = tf.distribute.MirroredStrategy([gpu.name for gpu in gpus])
  print('Running on multiple GPUs ', [gpu.name for gpu in gpus])
elif len(gpus) == 1:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('Running on single GPU ', gpus[0].name)
else:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('Running on CPU')
  
print("Number of accelerators: ", strategy.num_replicas_in_sync)

Running on single GPU  /device:GPU:0
Number of accelerators:  1


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/deepLearning
!ls

[Errno 2] No such file or directory: '/content/drive/MyDrive/deepLearning'
/content
drive  sample_data


## A)

Load dataset:

In [5]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

170498071/170498071 [==============================] - 13s 0us/step


Load pretrained model:

In [6]:
# Cifar-10 images are (32, 32). ResNet input is (224, 224). 
# As it's wanted to train on Cifar-10, I change model input size
# and keep images' size
IMG_SHAPE = (32, 32, 3)
resnet50_base = tf.keras.applications.ResNet50(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet',
                                               pooling='avg')
resnet50_base.trainable = False
preproc_input = tf.keras.applications.resnet50.preprocess_input
# Data augmentation to improve results
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.2)
])

94765736/94765736 [==============================] - 5s 0us/step


Add one Dense layer on top of base model:

In [12]:
inputs = tf.keras.Input(shape=IMG_SHAPE)
inputs = data_augmentation(inputs)
inputs = tf.cast(inputs, tf.float32)
x = preproc_input(inputs)
x = resnet50_base(x, training=False)
outputs = tf.keras.layers.Dense(10, activation='relu')(x)

teacher_model = tf.keras.Model(inputs, outputs)

In [13]:
teacher_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 tf.__operators__.getitem_1   (None, 32, 32, 3)        0         
 (SlicingOpLambda)                                               
                                                                 
 tf.nn.bias_add_1 (TFOpLambd  (None, 32, 32, 3)        0         
 a)                                                              
                                                                 
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense_1 (Dense)             (None, 10)                20490     
                                                                 
Total params: 23,608,202
Trainable params: 20,490
Non-train

Train model:

In [15]:
lr = 1e-4
teacher_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                      metrics=['accuracy'])

In [16]:
history = teacher_model.fit(x_train, y_train, epochs=50, batch_size=256, validation_data=(x_test, y_test))

Epoch 1/50
196/196 [==============================] - 10s 39ms/step - loss: 2.3527 - accuracy: 0.2332 - val_loss: 2.0083 - val_accuracy: 0.3305
Epoch 2/50
196/196 [==============================] - 7s 34ms/step - loss: 1.8381 - accuracy: 0.3981 - val_loss: 1.7182 - val_accuracy: 0.4507
Epoch 3/50
196/196 [==============================] - 7s 34ms/step - loss: 1.6393 - accuracy: 0.4787 - val_loss: 1.5907 - val_accuracy: 0.4980
Epoch 4/50
196/196 [==============================] - 7s 34ms/step - loss: 1.5057 - accuracy: 0.5313 - val_loss: 1.4743 - val_accuracy: 0.5443
Epoch 5/50
196/196 [==============================] - 7s 35ms/step - loss: 1.3896 - accuracy: 0.5700 - val_loss: 1.4025 - val_accuracy: 0.5707
Epoch 6/50
196/196 [==============================] - 7s 35ms/step - loss: 1.3064 - accuracy: 0.5937 - val_loss: 1.3447 - val_accuracy: 0.5884
Epoch 7/50
196/196 [==============================] - 7s 35ms/step - loss: 1.2415 - accuracy: 0.6124 - val_loss: 1.3019 - val_accuracy: 0.596

In [52]:
res = teacher_model.evaluate(x_test, y_test)

313/313 [==============================] - 6s 18ms/step - loss: 1.0077 - accuracy: 0.6675


## B)

In [17]:
class Distiller(tf.keras.Model):
  def __init__(self, student, teacher):
    super().__init__()
    self.teacher = teacher
    self.student = student
  
  def compile(self, optimizer, metrics, student_loss_fn, distillation_loss_fn, alpha=0.1, temperature=3):
    super().compile(optimizer=optimizer, metrics=metrics)
    self.student_loss_fn = student_loss_fn
    self.distillation_loss_fn = distillation_loss_fn
    self.alpha = alpha
    self.temperature = temperature

  def train_step(self, data):
    x, y = data
    teacher_preds = self.teacher(x, training=False)
    with tf.GradientTape() as tape:
      student_preds = self.student(x, training=True)
      student_loss = self.student_loss_fn(y, student_preds)

      distillation_loss = (
          self.distillation_loss_fn(
              tf.nn.softmax(teacher_preds/self.temperature, axis=1),
              tf.nn.softmax(student_preds/self.temperature, axis=1)
          ) * self.temperature**2
      )
      loss = self.alpha*student_loss + (1-self.alpha)*distillation_loss

    trainable_vars = self.student.trainable_variables
    grads = tape.gradient(loss, trainable_vars)

    self.optimizer.apply_gradients(zip(grads, trainable_vars))
    self.compiled_metrics.update_state(y, student_preds)

    results = {m.name: m.result() for m in self.metrics}
    results.update(
        {"student_loss": student_loss, "distillation_loss": distillation_loss}
    )
    return results

  def test_step(self, data):
    x, y = data

    y_preds = self.student(x, training=False)
    student_loss = self.student_loss_fn(y, y_preds)
    self.compiled_metrics.update_state(y, y_preds)

    results = {m.name: m.result() for m in self.metrics}
    results.update({"student_loss": student_loss})

    return results

To download Resnet18 model, I use `image-classifiers` package

In [18]:
!pip install image-classifiers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 KB 6.9 MB/s eta 0:00:00


In [19]:
from classification_models.tfkeras import Classifiers

In [20]:
resnet18, pre_input_resnet18 = Classifiers.get('resnet18')
resnet18_base = resnet18(IMG_SHAPE, weights=None, include_top=False)
# Note that I used None weights which means I only used the structure of resnet18

In [48]:
inputs = tf.keras.Input(shape=IMG_SHAPE)
inputs = tf.cast(inputs, tf.float32)
x = pre_input_resnet18(inputs)
x = resnet18_base(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(10, activation='relu')(x)
student_model = tf.keras.Model(inputs, outputs)

In [49]:
student_model.summary()

Model: "model_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_25 (InputLayer)       [(None, 32, 32, 3)]       0         
                                                                 
 model_2 (Functional)        (None, 1, 1, 512)         11186889  
                                                                 
 global_average_pooling2d_9   (None, 512)              0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_11 (Dense)            (None, 10)                5130      
                                                                 
Total params: 11,192,019
Trainable params: 11,184,077
Non-trainable params: 7,942
_________________________________________________________________


In [50]:
distiller = Distiller(student=student_model, teacher=teacher_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    metrics=['accuracy'],
    student_loss_fn=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    alpha=0.1,
    temperature=1
)

history = distiller.fit(x_train, y_train, epochs=20, batch_size=64, validation_data=(x_test, y_test))

Epoch 1/20
782/782 [==============================] - 33s 38ms/step - accuracy: 0.8169 - student_loss: 0.5722 - distillation_loss: 1.8864 - val_accuracy: 0.5649 - val_student_loss: 1.5239
Epoch 2/20
782/782 [==============================] - 29s 37ms/step - accuracy: 0.8422 - student_loss: 0.4375 - distillation_loss: 1.8425 - val_accuracy: 0.5695 - val_student_loss: 1.6436
Epoch 3/20
782/782 [==============================] - 29s 37ms/step - accuracy: 0.9084 - student_loss: 0.2688 - distillation_loss: 1.7957 - val_accuracy: 0.6304 - val_student_loss: 1.2489
Epoch 4/20
782/782 [==============================] - 28s 36ms/step - accuracy: 0.9273 - student_loss: 0.2130 - distillation_loss: 1.7785 - val_accuracy: 0.6282 - val_student_loss: 1.0816
Epoch 5/20
782/782 [==============================] - 28s 36ms/step - accuracy: 0.9209 - student_loss: 0.2203 - distillation_loss: 1.7754 - val_accuracy: 0.6293 - val_student_loss: 1.1666
Epoch 6/20
782/782 [==============================] - 29s 37

In [54]:
res = distiller.evaluate(x_test, y_test)

313/313 [==============================] - 2s 7ms/step - accuracy: 0.6320 - student_loss: 2.2720


## C) 
To explain differences fairly, same hyperparameters in part A) and B) are chosen for following parts.

In [55]:
inputs = tf.keras.Input(shape=IMG_SHAPE)
inputs = data_augmentation(inputs)
inputs = tf.cast(inputs, tf.float32)
x = pre_input_resnet18(inputs)
x = resnet18_base(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(10, activation='relu')(x)
scratch_student_model = tf.keras.Model(inputs, outputs)

In [56]:
lr = 1e-4
scratch_student_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                              metrics=['accuracy'])

In [57]:
scratch_student_model.summary()

Model: "model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_27 (InputLayer)       [(None, 32, 32, 3)]       0         
                                                                 
 model_2 (Functional)        (None, 1, 1, 512)         11186889  
                                                                 
 global_average_pooling2d_10  (None, 512)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_12 (Dense)            (None, 10)                5130      
                                                                 
Total params: 11,192,019
Trainable params: 11,184,077
Non-trainable params: 7,942
_________________________________________________________________


In [58]:
history = scratch_student_model.fit(x_train, y_train, epochs=20, batch_size=64, validation_data=(x_test, y_test))

Epoch 1/20
782/782 [==============================] - 24s 28ms/step - loss: 0.5816 - accuracy: 0.7862 - val_loss: 1.5573 - val_accuracy: 0.5862
Epoch 2/20
782/782 [==============================] - 19s 25ms/step - loss: 0.3242 - accuracy: 0.8741 - val_loss: 1.9305 - val_accuracy: 0.5870
Epoch 3/20
782/782 [==============================] - 19s 24ms/step - loss: 0.2992 - accuracy: 0.8807 - val_loss: 1.9448 - val_accuracy: 0.5735
Epoch 4/20
782/782 [==============================] - 20s 25ms/step - loss: 0.2841 - accuracy: 0.8853 - val_loss: 2.1087 - val_accuracy: 0.5783
Epoch 5/20
782/782 [==============================] - 19s 25ms/step - loss: 0.2877 - accuracy: 0.8831 - val_loss: 2.2144 - val_accuracy: 0.5794
Epoch 6/20
782/782 [==============================] - 19s 24ms/step - loss: 0.2785 - accuracy: 0.8868 - val_loss: 2.2247 - val_accuracy: 0.5762
Epoch 7/20
782/782 [==============================] - 19s 24ms/step - loss: 0.2744 - accuracy: 0.8880 - val_loss: 2.0821 - val_accuracy:

In [59]:
res = scratch_student_model.evaluate(x_test, y_test)

313/313 [==============================] - 2s 7ms/step - loss: 2.0888 - accuracy: 0.5866


ResNet18 model that is trained from scratch achieves $58.6\%$ accuracy on the test set while the same model that was trained with knowledge distillation achieved $63.2\%$ accuracy on the test set. The main reason is the teacher model in knowledge distillation scenario. In that case, teacher model helps student model to update its paramater such that it minimizes sum of two losses. This will prevent overvitting that appears in training ResNet18 model from scratch and results in better performance on the test set.

 ## D)
In this part first the ResNet50 model is trained from scratch. Then it's used as teacher model in knowledge distillation. Again, ResNet18 model is used as the student model. Note that ResNet50 is trained on ImageNet which consists of 1000 classes. So, FC layer must be removed and replaced.

In [60]:
resnet50_base_none = tf.keras.applications.ResNet50(input_shape=IMG_SHAPE,
                                                    include_top=False,
                                                    weights=None,
                                                    pooling='avg')

In [61]:
inputs = tf.keras.Input(shape=IMG_SHAPE)
inputs = data_augmentation(inputs)
inputs = tf.cast(inputs, tf.float32)
x = preproc_input(inputs)
x = resnet50_base_none(x, training=True)
outputs = tf.keras.layers.Dense(10, activation='relu')(x)

resnet50_scratch = tf.keras.Model(inputs, outputs)

In [62]:
resnet50_scratch.summary()

Model: "model_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_30 (InputLayer)       [(None, 32, 32, 3)]       0         
                                                                 
 tf.__operators__.getitem_2   (None, 32, 32, 3)        0         
 (SlicingOpLambda)                                               
                                                                 
 tf.nn.bias_add_2 (TFOpLambd  (None, 32, 32, 3)        0         
 a)                                                              
                                                                 
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense_13 (Dense)            (None, 10)                20490     
                                                                 
Total params: 23,608,202
Trainable params: 23,555,082
Non-

In [63]:
lr = 1e-4
resnet50_scratch.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                      metrics=['accuracy'])

In [64]:
history = resnet50_scratch.fit(x_train, y_train, epochs=50, batch_size=256, validation_data=(x_test, y_test))

Epoch 1/50
196/196 [==============================] - 83s 110ms/step - loss: 2.3215 - accuracy: 0.1395 - val_loss: 2.2689 - val_accuracy: 0.1574
Epoch 2/50
196/196 [==============================] - 19s 98ms/step - loss: 2.2564 - accuracy: 0.1731 - val_loss: 2.2251 - val_accuracy: 0.1986
Epoch 3/50
196/196 [==============================] - 19s 97ms/step - loss: 2.1910 - accuracy: 0.2199 - val_loss: 2.1711 - val_accuracy: 0.2362
Epoch 4/50
196/196 [==============================] - 19s 97ms/step - loss: 2.0739 - accuracy: 0.2750 - val_loss: 2.0463 - val_accuracy: 0.2870
Epoch 5/50
196/196 [==============================] - 19s 97ms/step - loss: 1.9482 - accuracy: 0.3262 - val_loss: 1.9746 - val_accuracy: 0.3155
Epoch 6/50
196/196 [==============================] - 19s 98ms/step - loss: 1.8050 - accuracy: 0.3693 - val_loss: 1.8922 - val_accuracy: 0.3313
Epoch 7/50
196/196 [==============================] - 19s 98ms/step - loss: 1.6717 - accuracy: 0.4108 - val_loss: 1.8355 - val_accuracy

In [67]:
res = resnet50_scratch.evaluate(x_test, y_test)

313/313 [==============================] - 6s 18ms/step - loss: 2.7490 - accuracy: 0.4683


Knowledge distillation:

In [65]:
inputs = tf.keras.Input(shape=IMG_SHAPE)
inputs = tf.cast(inputs, tf.float32)
x = pre_input_resnet18(inputs)
x = resnet18_base(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(10, activation='relu')(x)
student_model_with_resnet50_scratch = tf.keras.Model(inputs, outputs)

In [66]:
distiller = Distiller(student=student_model_with_resnet50_scratch, teacher=resnet50_scratch)
distiller.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    metrics=['accuracy'],
    student_loss_fn=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    alpha=0.1,
    temperature=1
)

history = distiller.fit(x_train, y_train, epochs=20, batch_size=64, validation_data=(x_test, y_test))

Epoch 1/20
782/782 [==============================] - 34s 38ms/step - accuracy: 0.7985 - student_loss: 0.5353 - distillation_loss: 1.7400 - val_accuracy: 0.6012 - val_student_loss: 1.0074
Epoch 2/20
782/782 [==============================] - 30s 38ms/step - accuracy: 0.8871 - student_loss: 0.2868 - distillation_loss: 1.7175 - val_accuracy: 0.6061 - val_student_loss: 2.3305
Epoch 3/20
782/782 [==============================] - 29s 37ms/step - accuracy: 0.8854 - student_loss: 0.2914 - distillation_loss: 1.7183 - val_accuracy: 0.5987 - val_student_loss: 1.9723
Epoch 4/20
782/782 [==============================] - 30s 38ms/step - accuracy: 0.8882 - student_loss: 0.2812 - distillation_loss: 1.7155 - val_accuracy: 0.5988 - val_student_loss: 1.7092
Epoch 5/20
782/782 [==============================] - 30s 38ms/step - accuracy: 0.8879 - student_loss: 0.2812 - distillation_loss: 1.7159 - val_accuracy: 0.5987 - val_student_loss: 1.8455
Epoch 6/20
782/782 [==============================] - 29s 38

In [68]:
res = distiller.evaluate(x_test, y_test)

313/313 [==============================] - 3s 8ms/step - accuracy: 0.6089 - student_loss: 2.3656


As you can see, the teacher accuracy is lower than teacher model in part A which was linear tuned. The reason is that to train such big network, more epochs must be passed and regularization techniques must be used. But, to compare results fairly, I used same hyperparameters which results in lower accuracy.

When this teacher is used, it's obvious that we're going to have lower accuracy compared to part C. In this scenario, student's accuracy is $60.89\%$ but in part C, student's accuracy was $63.2\%$. Although there is difference in accuracy, but due to student model itself, the difference is not that high. 